In [1]:
from utilities import *
from model.tripartite_lp import GNNPolicy as Trimodel
from model.rgcn4 import GNNPolicy as RGCNmodel
from torch_geometric.data import HeteroData
import collections
import gurobipy as gp
from uc_class import UC
import os
import json
import numpy as np
np.set_printoptions(threshold=np.inf)
#加入关机程序，要不违反爬坡，最小启停时间，spin约束
#效果可能有限，有时间再做


In [2]:
uc_path = '/home/peilun/predict_heuristicFix_loopEnhance/UC_AF/80_c11_based_8_std.mod'
uc = UC(uc_path)

In [3]:
tri_path = 'model_save/1bin_80_c11_based_8_std_tripartite_lpsolve/model_best_acc.pth'
tri_model = Trimodel()
tri_dict = torch.load(tri_path)
tri_model.load_state_dict(tri_dict)
tri_model.eval()
print()

GNNPolicy(
  (node_embedding): ModuleList(
    (0): Sequential(
      (0): LayerNorm((7,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=7, out_features=64, bias=True)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ReLU()
    )
    (1): Sequential(
      (0): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=4, out_features=64, bias=True)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ReLU()
    )
    (2): Sequential(
      (0): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=1, out_features=64, bias=True)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ReLU()
    )
  )
  (edge_embedding): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
  (con1): Sequential(
    (0): BipartiteGraphConvolution()
    (1): BipartiteGraphConvolution()
    (2): BipartiteGraphConvolution()
 

In [4]:
log=False
gaps=[]
json_test_path = 'datasets/80_c11_based_8_std/json/test'
json_test_files = os.listdir(json_test_path)[:100]

shut_down = uc.Pishutdown
startup = uc.Pistartup
on = uc.ThTime_on_min
off = uc.ThTime_off_min
pmax = uc.ThPimax
pmin = uc.ThPimin
ramp_up = uc.Piup
ramp_down = uc.Pidown

import time
for file in json_test_files:
    print(file)
    os.path.join(json_test_path, file)
    with open(os.path.join(json_test_path, file),'r') as f:
                data = json.load(f)

    begin = time.time()
    pmax = uc.ThPimax
    
    Dt = data['Dt']
    # Spin = [a + b for a, b in zip(data['Spin'], data['Dt'])]
    Spin = data['Spin']
    u0 = data['u0']
    p0 = data['p0']
    on_off = data['on_off']
    m1 = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
    # m1.write('test1.lp')
    # print(on_off[11],on[11],off[11])
    t1=time.time()
    if log:
        print('===================================')
        print('uc建模',t1-begin)
    
    graph=get_Tripartite_graph_lp(m1)
    t2=time.time()
    if log:
        print('图建模',t2-t1)
    sol = tri_model(graph).detach().cpu().numpy().reshape(-1,24)

    N=sol.shape[0]
    T=sol.shape[1]
    t3=time.time()
    if log:
        print('gnn预测',t3-t2)

    sol = restore_initial(sol,p0,ramp_down,shut_down,on_off,on,off)


    sol[sol >= 0.9] = 1
    sol = restore_spin(sol,Spin,Dt,pmax)
    
    # must_close = (sol == -1).astype(int)
    
    # sol[(sol < 1-(1e-5)) & (sol > -1e-5)] = 0
    # sol[sol < 1-(1e-5)] = 0
    sol = restore_on_off(sol,on_off,on,off)
    t4=time.time()
    if log:
        print('修复ramp以外约束',t4-t3)
    # x=np.array([1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,0,0,0]).reshape(1,24)
    # r2l,l2r = com_cont_on_off(x,on_off[:1],off[:1])
    # print(x)
    # print(on_off[:1])
    # print(off[:1])
    # print(l2r)
    # print(r2l)
    sol = restore_ramp(sol,Dt,u0,p0,ramp_down,ramp_up,pmin,pmax,startup,shut_down,on_off,off)
    sol = restore_on_off(sol,on_off,on,off)
    
    #加入关机程序，要不违反爬坡，最小启停时间，spin约束
    #效果可能有限，有时间再做

    # sol[sol < 1-(1e-5)] = 0


    t5=time.time()
    if log:
        print('ramp子问题',t5-t4)
        print("************")
        print('总时间',t5-begin)
        print('减去建模时间',t5-t2)
    sol[sol < 1-(1e-5)] = 0
    u = m1.getVars()[:N*T]
    p = m1.getVars()[N*T:N*T*2]
    for i in range(N):
        for t in range(T):
            m1.addConstr(u[i*T+t]==sol[i,t],name=f'fix_u[{i},{t}]')


    # m1.setParam('OutputFlag', 1)
    # m1.write('test.lp')
            
    m1.optimize()
    print('最优解gap',(m1.objVal-data['obj'])/(data['obj']+1e-5))
    gaps.append((m1.objVal-data['obj'])/(data['obj']+1e-5))
    #把u的值写到csv文件里面
    # sub_opt_u = np.array([u[i].x for i in range(N*T)]).reshape(N,T)
    # np.savetxt('sub_opt_u.csv',sub_opt_u,delimiter=',')
    # sub_opt_p = np.array([p[i].x for i in range(N*T)]).reshape(N,T)
    # np.savetxt('sub_opt_p.csv',sub_opt_p,delimiter=',')
    # opt_u=np.array(data['opt_sol']).reshape(N,T)
    # np.savetxt('opt_u.csv',opt_u,delimiter=',')


    # m1.computeIIS()
    # for c in m1.getConstrs():
    #     if c.IISConstr:
    #         print(c.constrName)
    # print((m1.objVal-data['obj'])/(data['obj']+1e-5))
print('平均gap',np.mean(gaps))

107_0.json
最优解gap 0.0017547395214096556
107_1.json
最优解gap 0.014022274877014651
107_10.json
最优解gap 0.0013919178305016636
107_11.json
最优解gap 0.003084966047032785
107_12.json
最优解gap 0.006343778114415989
107_13.json
最优解gap 0.0035565151481193997
107_14.json
最优解gap 0.005581927591023668
107_15.json
最优解gap 0.003168911089930808
107_16.json
最优解gap 0.003395883375131446
107_17.json
最优解gap 0.002700276366947018
107_18.json
最优解gap 0.01672159322310305
107_19.json
最优解gap 0.01319486403261473
107_2.json
最优解gap 0.022213839308645424
107_3.json
最优解gap 0.00868430188011676
107_4.json
最优解gap 0.004075467670176599
107_5.json
最优解gap 0.004501873633296918
107_6.json
最优解gap 0.01909075472517623
107_7.json
最优解gap 0.0033124183553173937
107_8.json
最优解gap 0.0027449407281382107
107_9.json
最优解gap 0.006596412126871024
117_0.json
最优解gap 0.009477021455685396
117_1.json
最优解gap 0.008077499147677063
117_10.json
最优解gap 0.00947912398821271
117_11.json
最优解gap 0.009148748295729503
117_12.json
最优解gap 0.007815142295530741
117_13.json


In [5]:
# json_test_path = 'datasets/80_c11_based_8_std/json/test'
# json_test_files = os.listdir(json_test_path)[:100]
gaps=[]
rd = uc.Pidown
sd = uc.Pishutdown
on = uc.ThTime_on_min
off = uc.ThTime_off_min
pmax = uc.ThPimax
pmin = uc.ThPimin
ramp_up = uc.Piup
ramp_down = uc.Pidown

import time
begin = time.time()
for file in json_test_files:
    os.path.join(json_test_path, file)
    with open(os.path.join(json_test_path, file),'r') as f:
                data = json.load(f)
    pmax = uc.ThPimax
    
    Dt = data['Dt']
    Spin = data['Spin']
    u0 = data['u0']
    p0 = data['p0']
    on_off = data['on_off']
    m1 = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
    
    sol = np.array(data['weighted_sol']).reshape(-1,24)

    sol = restore_initial(sol,p0,rd,sd,on_off,on,off)
    
    sol[sol >= 0.99] = 1
    sol = restore_spin(sol,Spin,Dt,pmax)
    sol = restore_on_off(sol,on_off,on,off)
    sol = restore_ramp(sol,Dt,u0,p0,ramp_down,ramp_up,pmin,pmax,startup,shut_down,on_off,off)
    sol = restore_on_off(sol,on_off,on,off)
    sol[sol < 1-(1e-5)] = 0
    u = m1.getVars()[:N*T]
    p = m1.getVars()[N*T:N*T*2]
    for i in range(N):
        for t in range(T):
            m1.addConstr(u[i*T+t]==sol[i,t],name=f'fix_u[{i},{t}]')


    # m1.setParam('OutputFlag', 1)
    # m1.write('test.lp')
            
    m1.optimize()
    print('最优解gap',(m1.objVal-data['obj'])/(data['obj']+1e-5))
    gaps.append((m1.objVal-data['obj'])/(data['obj']+1e-5))
print('平均gap',np.mean(gaps))

    

最优解gap 0.0006964153136931156
最优解gap 1.2398501519192884e-14
最优解gap 0.000512349562008828
最优解gap 0.0006143951954374428
最优解gap 0.00017485221148419712
最优解gap 0.006487278742638879
最优解gap 0.0009781293610070679
最优解gap 6.490891253191844e-16
最优解gap 0.006494912055781996
最优解gap 8.12178122410223e-15
最优解gap 0.00033963777627422296
最优解gap 0.003747666024556279
最优解gap 0.008355507172487696
最优解gap 0.004437524284498792
最优解gap 0.0009621357293934388
最优解gap 0.0005137928169560656
最优解gap 0.0
最优解gap 5.786293045006034e-15
最优解gap 0.004987526050664143
最优解gap 0.0008418015475711061
最优解gap 0.0021133214367414035
最优解gap 0.001511505838204593
最优解gap 0.00702126150565601
最优解gap 0.0021787149658000575
最优解gap 0.0043498522858988215
最优解gap 0.0016660329611426993
最优解gap 1.597085064636372e-15
最优解gap 0.005145782890338907
最优解gap 0.0005556893551226222
最优解gap 0.002713974858743903
最优解gap 0.0005222436226050253
最优解gap 0.004314705590466974
最优解gap 0.005212011345098429
最优解gap 0.006606293158757326
最优解gap 0.004938185698382383
最优解gap 0.00579851

In [6]:
# json_test_path = 'datasets/80_c11_based_8_std/json/test'
# json_test_files = os.listdir(json_test_path)[:100]
gaps=[]
rd = uc.Pidown
sd = uc.Pishutdown
on = uc.ThTime_on_min
off = uc.ThTime_off_min
pmax = uc.ThPimax
pmin = uc.ThPimin
ramp_up = uc.Piup
ramp_down = uc.Pidown

import time
begin = time.time()
for file in json_test_files:
    os.path.join(json_test_path, file)
    with open(os.path.join(json_test_path, file),'r') as f:
                data = json.load(f)
    pmax = uc.ThPimax
    
    Dt = data['Dt']
    Spin = data['Spin']
    u0 = data['u0']
    p0 = data['p0']
    on_off = data['on_off']
    m1 = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
    r=m1.relax()
    r.setParam('OutputFlag', 0)
    r.optimize()
    u=r.getVars()[:len(u0)*len(Dt)]

    sol = np.array([u[i].x for i in range(len(u))]).reshape(len(u0),len(Dt))

    sol = restore_initial(sol,p0,rd,sd,on_off,on,off)
    
    sol[sol >= 0.99] = 1
    sol = restore_spin(sol,Spin,Dt,pmax)
    sol = restore_on_off(sol,on_off,on,off)
    sol = restore_ramp(sol,Dt,u0,p0,ramp_down,ramp_up,pmin,pmax,startup,shut_down,on_off,off)
    sol = restore_on_off(sol,on_off,on,off)
    t5=time.time()
    sol[sol < 1-(1e-5)] = 0
    u = m1.getVars()[:N*T]
    p = m1.getVars()[N*T:N*T*2]
    for i in range(N):
        for t in range(T):
            m1.addConstr(u[i*T+t]==sol[i,t],name=f'fix_u[{i},{t}]')


    # m1.setParam('OutputFlag', 1)
    # m1.write('test.lp')
            
    m1.optimize()
    print('最优解gap',(m1.objVal-data['obj'])/(data['obj']+1e-5))
    gaps.append((m1.objVal-data['obj'])/(data['obj']+1e-5))
print('平均gap',np.mean(gaps))

    

最优解gap 0.0064723803427092705
最优解gap 0.009868684510556312
最优解gap 0.011855957447305639
最优解gap 0.009036524534491669
最优解gap 0.0047040140751168635
最优解gap 0.001489419428544634
最优解gap 0.00232773976656682
最优解gap 0.0030953824280172937
最优解gap 0.008568976966898158
最优解gap 0.0035521139703658506
最优解gap 0.00447922702759131
最优解gap 0.012857856204174446
最优解gap 0.008751810540819046
最优解gap 0.006246867009474461
最优解gap 0.002212218582343402
最优解gap 0.011885810402680056
最优解gap 0.0108434211835327
最优解gap 0.017387935529238802
最优解gap 0.01006010066783101
最优解gap 0.007051122962813135
最优解gap 0.010304951686217275
最优解gap 0.009906197228747967
最优解gap 0.005504822463794814
最优解gap 0.010629404324909796
最优解gap 0.008484180970471916
最优解gap 0.009439972531452309
最优解gap 0.00772100465508035
最优解gap 0.00929426406009436
最优解gap 0.010086241616757556
最优解gap 0.008715907291008095
最优解gap 0.00887748718434162
最优解gap 0.0064389691180383995
最优解gap 0.011004379224437536
最优解gap 0.010740698052972229
最优解gap 0.00926042164827332
最优解gap 0.008549957213276